Aqui está um código simples com sklearn para testar a execução de Python no Codespace com uma Decision Tree:

In [1]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.tree import plot_tree
import joblib
import xgboost as xgb
import pickle

In [3]:
def carregar_dataset(dataset_func):
    """
    Descrição:
        Essa função é usada para carregar um dataset do Scikit-learn, separando-o em dados e rótulos.

    Parâmetros:
        dataset_func: função
            Função para carregar o dataset (por exemplo, `load_iris`, `load_digits`).

    Retorno:
        dataset: objeto
            O dataset completo carregado.
        X: array
            As características (dados) do dataset.
        y: array
            Os rótulos do dataset.
    """
    # Carregar o dataset especifico
    dataset = dataset_func()
    X = dataset.data
    Y = dataset.target

    return dataset, X, Y

In [5]:
def dividir_treinamento(X, Y, test_size=0.2, random_state=42):
    """"
    Descrição: 
        Função usada para Dividir matrizes ou matrizes em subconjuntos de trem aleatório e teste. para isso vou usar a função train_test_split()
        
    Parametrôs: data, label, test_size, random_state
        X = Características de entrada
        Y = Rotulos
        test_size = representar a proporção do conjunto de dados para incluir na divisão de teste.
        random_state = Controla o embaralhamento aplicado aos dados antes de aplicar a divisão. Passe um int para saída reprodutível em várias chamadas de função.

    Retorno: X_train, X_test, y_train, y_test
        X_train = dados de treinamento separado
        X_test = dados de teste separado
        y_train = rotulos de treinamento
        y_test = rotulos de teste
    """
    # Dividir o dataset em treinamento e teste
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=random_state)

    return X_train, X_test, y_train, y_test

In [4]:
def normalizar_dados(X_train, X_test):
    """
    Descrição:
        Função para normalizar os dados de treinamento e teste usando StandardScaler.
    
    Parâmetros:
        X_train: array-like
            Dados de treinamento.
        X_test: array-like
            Dados de teste.
    
    Retorno:
        X_train_normalizado: array-like
            Dados de treinamento normalizados.
        X_test_normalizado: array-like
            Dados de teste normalizados.
        scaler: StandardScaler
            O objeto StandardScaler treinado, que pode ser usado para normalizar novos dados no futuro.
    """
    # Instanciando o StandardScaler
    scaler = StandardScaler()
    
    # Ajustando o scaler e transformando os dados de treinamento
    X_train_normalizado = scaler.fit_transform(X_train)
    
    # Transformando os dados de teste com o mesmo scaler
    X_test_normalizado = scaler.transform(X_test)
    
    return X_train_normalizado, X_test_normalizado, scaler

In [6]:
def treinando_modelo(modelo, X_train, y_train, X_test, random_state=None):
    """
    Descrição:
        Função para treinar um modelo de classificação usando o sklearn.
    
    Parâmetros:
        modelo: classificador
            O modelo que queremos usar como classificador (por exemplo, DecisionTreeClassifier, RandomForestClassifier).
        X_train: array-like
            Dados de treinamento.
        X_test: array-like
            Dados de teste.
        y_train: array-like
            Rótulos do conjunto de treinamento.
        random_state: int, opcional
            Controla o embaralhamento aplicado aos dados antes de aplicar a divisão. Passe um int para saída reprodutível.

    Retorno:
        modelo_treinado: classificador
            O modelo treinado.
        y_pred: array
            Previsões do modelo para o conjunto de teste.
    """
    # Verificar se o modelo aceita o parâmetro random_state
    if 'random_state' in modelo().get_params().keys():
        modelo_treinado = modelo(random_state=random_state)
    else:
        modelo_treinado = modelo()

    # Treinar o modelo
    modelo_treinado.fit(X_train, y_train)

    # Prever as classes para o conjunto de teste
    y_pred = modelo_treinado.predict(X_test)

    return modelo_treinado, y_pred

In [7]:
def treinando_com_grid_search(modelo, param_grid, X_train, y_train, X_test, cv=5, random_state=42):
    """
    Descrição:
        Função para treinar um modelo de classificação usando GridSearchCV para encontrar os melhores hiperparâmetros.
    
    Parâmetros:
        modelo: classificador
            O modelo que queremos usar como classificador (por exemplo, DecisionTreeClassifier, RandomForestClassifier).
        param_grid: dict
            Dicionário contendo os hiperparâmetros que serão testados no GridSearchCV.
        X_train: array-like
            Dados de treinamento.
        y_train: array-like
            Rótulos do conjunto de treinamento.
        X_test: array-like
            Dados de teste.
        cv: int, opcional (default=5)
            Número de divisões para a validação cruzada.
        random_state: int, opcional (default=42)
            Controla o embaralhamento aplicado aos dados para garantir reprodutibilidade.

    Retorno:
        melhor_modelo: classificador
            O melhor modelo treinado com os hiperparâmetros otimizados.
        y_pred: array
            Previsões do modelo para o conjunto de teste.
        melhores_parametros: dict
            Os melhores hiperparâmetros encontrados pelo GridSearchCV.
    """
    # Instanciando o GridSearchCV
    grid_search = GridSearchCV(estimator=modelo(random_state=random_state), param_grid=param_grid, cv=cv, n_jobs=-1, verbose=2)
    
    # Executando o GridSearchCV para encontrar os melhores hiperparâmetros
    grid_search.fit(X_train, y_train)
    
    # Obter os melhores parâmetros
    melhores_parametros = grid_search.best_params_
    
    # Treinar o modelo final com os melhores parâmetros
    melhor_modelo = grid_search.best_estimator_
    
    # Prever as classes para o conjunto de teste
    y_pred = melhor_modelo.predict(X_test)
    
    return melhor_modelo, y_pred, melhores_parametros

In [8]:
def treinando_com_grid_search(modelo, param_grid, X_train, y_train, X_test, cv=5, random_state=42):
    """
    Descrição:
        Função para treinar um modelo de classificação usando GridSearchCV para encontrar os melhores hiperparâmetros.
    
    Parâmetros:
        modelo: classificador
            O modelo que queremos usar como classificador (por exemplo, DecisionTreeClassifier, RandomForestClassifier).
        param_grid: dict
            Dicionário contendo os hiperparâmetros que serão testados no GridSearchCV.
        X_train: array-like
            Dados de treinamento.
        y_train: array-like
            Rótulos do conjunto de treinamento.
        X_test: array-like
            Dados de teste.
        cv: int, opcional (default=5)
            Número de divisões para a validação cruzada.
        random_state: int, opcional (default=42)
            Controla o embaralhamento aplicado aos dados para garantir reprodutibilidade.

    Retorno:
        melhor_modelo: classificador
            O melhor modelo treinado com os hiperparâmetros otimizados.
        y_pred: array
            Previsões do modelo para o conjunto de teste.
        melhores_parametros: dict
            Os melhores hiperparâmetros encontrados pelo GridSearchCV.
    """
    # Instanciando o GridSearchCV
    grid_search = GridSearchCV(estimator=modelo(random_state=random_state), param_grid=param_grid, cv=cv, n_jobs=-1, verbose=2)
    
    # Executando o GridSearchCV para encontrar os melhores hiperparâmetros
    grid_search.fit(X_train, y_train)
    
    # Obter os melhores parâmetros
    melhores_parametros = grid_search.best_params_
    
    # Treinar o modelo final com os melhores parâmetros
    melhor_modelo = grid_search.best_estimator_
    
    # Prever as classes para o conjunto de teste
    y_pred = melhor_modelo.predict(X_test)
    
    return melhor_modelo, y_pred, melhores_parametros


In [9]:
def configurar_e_treinar_modelo_arvore_decisao(X_train, X_test, y_train, max_depth=None, min_samples_split=2, min_samples_leaf=1, max_features=None, random_state=42):
    """
    Descrição:
        Função para configurar e treinar um modelo de árvore de decisão usando o conjunto de dados de treinamento.

    Parâmetros:
        X_train: array-like
            Dados de treinamento.
        y_train: array-like
            Rótulos do conjunto de treinamento.
        max_depth: int ou None, opcional (default=None)
            Profundidade máxima da árvore. Se None, os nós são expandidos até todas as folhas serem puras.
        min_samples_split: int, opcional (default=2)
            O número mínimo de amostras necessárias para dividir um nó.
        min_samples_leaf: int, opcional (default=1)
            O número mínimo de amostras necessárias para estar em um nó folha.
        max_features: int, float, string ou None, opcional (default=None)
            O número de características a considerar ao procurar a melhor divisão.
        random_state: int, opcional (default=42)
            Controla o embaralhamento aplicado aos dados para garantir reprodutibilidade.

    Retorno:
        modelo_treinado: classificador
            O modelo treinado de árvore de decisão.
    """
    # Configurando o modelo de árvore de decisão
    modelo = DecisionTreeClassifier(
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        random_state=random_state
    )

    # Treinando o modelo
    modelo_treinado = modelo.fit(X_train, y_train)

    y_pred = modelo_treinado.predict(X_test)

    return modelo_treinado, y_pred

In [10]:
def treinar_modelo_ensemble(X_train, y_train, X_test, n_estimators=100, max_depth=None, random_state=42):
    """
    Descrição:
        Função para treinar um modelo Ensemble, especificamente um RandomForestClassifier, usando o conjunto de dados de treinamento.
    
    Parâmetros:
        X_train: array-like
            Dados de treinamento.
        y_train: array-like
            Rótulos do conjunto de treinamento.
        X_test: array-like
            Dados de teste.
        n_estimators: int, opcional (default=100)
            O número de árvores na floresta.
        max_depth: int ou None, opcional (default=None)
            Profundidade máxima da árvore. Se None, as árvores crescem até todas as folhas serem puras ou até que todas as folhas contenham menos que min_samples_split amostras.
        random_state: int, opcional (default=42)
            Controla o embaralhamento aplicado às árvores e amostras para garantir reprodutibilidade.

    Retorno:
        modelo_treinado: classificador
            O modelo Ensemble treinado.
        y_pred: array
            Previsões do modelo para o conjunto de teste.
    """
    # Configurando o modelo Ensemble - RandomForestClassifier
    modelo = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=random_state
    )

    # Treinando o modelo
    modelo_treinado = modelo.fit(X_train, y_train)

    # Fazendo previsões
    y_pred = modelo_treinado.predict(X_test)

    return modelo_treinado, y_pred

In [11]:
def train_xgboost(X_train, y_train, X_test, y_test, max_depth=10, n_estimators=100, learning_rate=0.1, random_state=42):
    """
    Treina um modelo XGBoost.
    
    Argumentos:
        X_train: array numpy com os dados de treinamento.
        y_train: array numpy com as labels de treinamento.
        X_test: array numpy com os dados de test.
        y_test: array numpy com as labels de test.
        max_depth: profundidade máxima das árvores.
        n_estimators: número de árvores na floresta.
        learning_rate: taxa de aprendizado.
        
    Retorna:
        modelo: o classificador XGBoost treinado.
    """
    model_xgboost = xgb.XGBClassifier(max_depth=max_depth, n_estimators=n_estimators, learning_rate=learning_rate, random_state=random_state)
    model_xgboost.fit(X_train, y_train)

    y_pred_xgboost = model_xgboost.predict(X_test)
    
    return model_xgboost, y_pred_xgboost

In [12]:
def calcular_metricas(y_test, y_pred):
    """
    Descrição:
        Função para calcular as métricas de precisão, recall e F1-Score.
    
    Parâmetros:
        y_test: array-like
            Rótulos reais do conjunto de teste.
        y_pred: array-like
            Rótulos preditos pelo modelo.
    
    Retorno:
        precision: float
            A precisão ponderada do modelo.
        recall: float
            O recall ponderado do modelo.
        f1: float
            O F1-Score ponderado do modelo.
    """
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    return precision, recall, f1

In [13]:
def plotar_comparacao_metricas_linhas(labels, decision_tree_scores, ensemble_scores, xgboost_scores):
    """
    Descrição:
        Função para plotar a comparação das métricas entre o modelo de Árvore de Decisão e o modelo Ensemble usando um gráfico de linhas.
    
    Parâmetros:
        labels: list
            Lista de rótulos para as métricas ('Precision', 'Recall', 'F1-Score').
        decision_tree_scores: list
            Lista de métricas do modelo de Árvore de Decisão.
        ensemble_scores: list
            Lista de métricas do modelo Ensemble.
    """
    x = np.arange(len(labels))  # Índices das métricas
    
    # Criando o gráfico de linhas
    plt.plot(x, decision_tree_scores, marker='o', linestyle='-', label='Decision Tree')
    plt.plot(x, ensemble_scores, marker='o', linestyle='-', label='Ensemble (Random Forest)')
    plt.plot(x, xgboost_scores, marker='o', linestyle='-', label='xgboost')
    
    # Adicionando detalhes ao gráfico
    plt.xticks(ticks=x, labels=labels)
    plt.ylabel('Scores')
    plt.title('Comparação das Métricas entre Decision Tree, Ensemble e xgboost')
    plt.legend()
    plt.grid(True)
    
    # Exibir o gráfico
    plt.show()

In [14]:
def visualizar_arvore(modelo, feature_names=None, class_names=None):
    """
    Descrição:
        Função para visualizar a árvore de decisão treinada.
    
    Parâmetros:
        modelo: classificador
            O modelo de árvore de decisão treinado.
        feature_names: list, opcional
            Lista com os nomes das características (features).
        class_names: list, opcional
            Lista com os nomes das classes.
    """
    plt.figure(figsize=(20, 10))
    plot_tree(modelo, feature_names=feature_names, class_names=class_names, filled=True)
    plt.show()

In [15]:
def calculate_gini(p):
    """
    Calcula o índice de Gini.
    
    Argumento:
        p: lista de proporções de classes no nó.
        
    Retorna:
        gini: valor da impureza Gini.
    """
    return 1 - sum([pi**2 for pi in p])

In [16]:
def calculate_entropy(p):
    """
    Calcula a entropia.
    
    Argumento:
        p: lista de proporções de classes no nó.
        
    Retorna:
        entropy: valor da entropia.
    """
    return -sum([pi * np.log2(pi) for pi in p if pi > 0])

In [17]:
def impureza_no_raiz(modelo, X, y):
    """
    Descrição:
        Calcula a impureza de Gini no nó raiz de um modelo de árvore de decisão.
    
    Parâmetros:
        modelo: classificador
            O modelo de árvore de decisão treinado.
        X: array-like
            Conjunto de dados usado para o treinamento.
        y: array-like
            Rótulos associados ao conjunto de dados.
    
    Retorno:
        impureza_gini: float
            A impureza de Gini no nó raiz.
    """
    # Pegar as amostras que chegaram ao nó raiz
    node_indicator = modelo.decision_path(X)
    node_index = node_indicator.indices[node_indicator.indptr[0]:node_indicator.indptr[1]]
    
    # Amostras no nó raiz
    classes_no_raiz = y[node_index]
    
    # Calcular a impureza de Gini
    impureza_gini = calculate_gini(classes_no_raiz)
    
    return impureza_gini

Exercício 1 - Treinamento do modelo baseado em árvore de decisão

Neste exercício, você irá treinar um modelo de árvore de decisão utilizando o conjunto de dados MNIST. O objetivo é entender como funciona o processo de treinamento de um modelo de árvore de decisão e como ajustar os parâmetros para melhorar o desempenho do modelo.

Você irá:

Carregar o conjunto de dados MNIST e pré-processar os dados, se necessário.
Selecionar a biblioteca ou framework que você irá utilizar (por exemplo, Scikit-learn e/ou XGBoost).
Configurar os parâmetros do modelo, como a profundidade da árvore, o número de características a considerar em cada nó, etc.
Treinar o modelo utilizando o conjunto de dados de treinamento.

In [18]:
#Exercicio 1
dataset, X, Y = carregar_dataset(load_digits)
# Agora, você pode usar `X` e `y` em outras partes do seu código
print("Características (X):", X[:5])
print("Rótulos (y):", Y[:5])
#dividindo o treinamento
X_train, X_test, y_train, y_test = dividir_treinamento(X, Y)

#normalizando os dados
X_train_normalizado, X_test_normalizado, scaler = normalizar_dados(X_train, X_test)

print("dados normalizados de treinamento", X_train_normalizado)
print("dados normalizados de teste", X_test_normalizado)

Características (X): [[ 0.  0.  5. 13.  9.  1.  0.  0.  0.  0. 13. 15. 10. 15.  5.  0.  0.  3.
  15.  2.  0. 11.  8.  0.  0.  4. 12.  0.  0.  8.  8.  0.  0.  5.  8.  0.
   0.  9.  8.  0.  0.  4. 11.  0.  1. 12.  7.  0.  0.  2. 14.  5. 10. 12.
   0.  0.  0.  0.  6. 13. 10.  0.  0.  0.]
 [ 0.  0.  0. 12. 13.  5.  0.  0.  0.  0.  0. 11. 16.  9.  0.  0.  0.  0.
   3. 15. 16.  6.  0.  0.  0.  7. 15. 16. 16.  2.  0.  0.  0.  0.  1. 16.
  16.  3.  0.  0.  0.  0.  1. 16. 16.  6.  0.  0.  0.  0.  1. 16. 16.  6.
   0.  0.  0.  0.  0. 11. 16. 10.  0.  0.]
 [ 0.  0.  0.  4. 15. 12.  0.  0.  0.  0.  3. 16. 15. 14.  0.  0.  0.  0.
   8. 13.  8. 16.  0.  0.  0.  0.  1.  6. 15. 11.  0.  0.  0.  1.  8. 13.
  15.  1.  0.  0.  0.  9. 16. 16.  5.  0.  0.  0.  0.  3. 13. 16. 16. 11.
   5.  0.  0.  0.  0.  3. 11. 16.  9.  0.]
 [ 0.  0.  7. 15. 13.  1.  0.  0.  0.  8. 13.  6. 15.  4.  0.  0.  0.  2.
   1. 13. 13.  0.  0.  0.  0.  0.  2. 15. 11.  1.  0.  0.  0.  0.  0.  1.
  12. 12.  1.  0.  0.  0.  0.  0.  1

In [20]:
# Definindo o espaço de hiperparâmetros para o GridSearchCV
param_grid = {
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

# Chamando a função treinando_com_grid_search
modelo_DecisionTree_gridseach, y_pred_DecisionTree_gridseach, melhores_parametros = treinando_com_grid_search(DecisionTreeClassifier, param_grid, X_train_normalizado, y_train, X_test_normalizado)
joblib.dump(modelo_DecisionTree_gridseach, '../model/modelo_DecisionTree_gridseach.pkl')
# Exibindo os melhores parâmetros encontrados
print("Melhores parâmetros encontrados:", melhores_parametros)

# Exibindo o relatório de classificação
print("Relatório de Classificação:")
print(classification_report(y_test, y_pred_DecisionTree_gridseach))
print("Precisão:", modelo_DecisionTree_gridseach.score(X_test_normalizado, y_test))

Fitting 5 folds for each of 72 candidates, totalling 360 fits
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1, min_samples_split=5; total time=   0.0s
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1, min_samples_split=5; total time=   0.0s
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1, min_samples_split=5; total time=   0.0s
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1, min_samples_split=5; total time=   0.0s
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1

FileNotFoundError: [Errno 2] No such file or directory: '../model/modelo_DecisionTree_gridseach.pkl'